# Cost for gate infidelity and parameter for Adam optimizer. And setup both for manual and Auto-grad are same

In [3]:

from qoc import grape_schroedinger_discrete
from qoc.standard import (Adam,TargetStateInfidelity,TargetStateInfidelityTime,
                          conjugate_transpose,matrix_to_column_vector_list,
                          get_annihilation_operator,
                          get_creation_operator,
                          SIGMA_Z,
                          generate_save_file_path,)
from qoc.models.operationpolicy import OperationPolicy
from qoc.models import (Dummy, EvolveSchroedingerDiscreteState,
                        EvolveSchroedingerResult,
                        GrapeSchroedingerDiscreteState,
                        GrapeSchroedingerResult,
                        InterpolationPolicy,
                        MagnusPolicy,
                        ProgramType,)
import numpy as np

# Define the system.
HILBERT_SIZE = 2
ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
# E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
sigmax=ANNIHILATION_OPERATOR +CREATION_OPERATOR
sigmay=-1j*ANNIHILATION_OPERATOR+1j*CREATION_OPERATOR
# E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
H_SYSTEM_0 = SIGMA_Z / 2
# Only real control amplitutdes are supported!
hamiltonian = lambda controls, time: (H_SYSTEM_0
                                      + controls[0] * sigmax
                                      + controls[1]*sigmay)


# Define the optimization.
COMPLEX_CONTROLS = False
CONTROL_COUNT = 2
EVOLUTION_TIME = 10 # nanoseconds
CONTROL_EVAL_COUNT = SYSTEM_EVAL_COUNT = EVOLUTION_TIME + 1
ITERATION_COUNT = 1000

# Define output.
LOG_ITERATION_STEP = 1
SAVE_ITERATION_STEP = 1
SAVE_PATH = "./out"
SAVE_FILE_NAME = "transmon_pi"
SAVE_FILE_PATH = generate_save_file_path(SAVE_FILE_NAME, SAVE_PATH)
INITIAL_STATES = matrix_to_column_vector_list(np.eye(2))
# we could have equivalently done
# initial_state0 = np.array([[1], [0]])
# initial_state1 = np.array([[0], [1]])
# initial_states = np.stack((initial_state0, initial_state1))
target_unitary = np.array([[0, 1], [1, 0]])
target_states = matrix_to_column_vector_list(target_unitary)
# we could have equivalently done
# target_state0 = np.array([[0], [1]])
# target_state1 = np.array([[1], [0]])
# target_states = np.stack((target_state0, target_state1))


#cost_multiplier is hyperprameter of each cost function
#Set neglect_relative_phase as True, relative phase of each column of gate U will be ignored.
#cost_eval_step :: int >= 1- This value determines how often step-costs are evaluated.The units of this value are in system_eval steps. E.g. if this value is 2,step-costs will be computed every 2 system_eval steps.

cost_1=TargetStateInfidelityTime(system_eval_count=SYSTEM_EVAL_COUNT,
                                                     target_states=target_states,cost_multiplier=1,
                                                     neglect_relative_phase=False,cost_eval_step=1)
cost_2=TargetStateInfidelity(target_states=target_states,cost_multiplier=1,neglect_relative_phase=False)
COSTS = [cost_1]
CONTROL_HAMILTONIAN=[sigmax,sigmay]
manual_parameter={"control_hamiltonian":CONTROL_HAMILTONIAN,"manual_gradient_mode":False}


optimizer=Adam(beta_1=0.9, beta_2=0.999, clip_grads=None,
                 epsilon=1e-8, learning_rate=1e-3,
                 learning_rate_decay=None, operation_policy=OperationPolicy.CPU,
                 scale_grads=None)
# a class to define the Adam optimizer
#     This implementation follows the original algorithm
#     https://arxiv.org/abs/1412.6980.
#     Fields:
#     apply_clip_grads :: bool - see clip_grads
#     apply_learning_rate_decay :: bool - see learning_rate_decay
#     apply_scale_grads :: bool - see scale_grads
#     beta_1 :: float - gradient decay bias
#     beta_2 :: float - gradient squared decay bias
#     clip_grads :: float - the maximum absolute value at which the gradients
#         should be element-wise clipped, if not set, the gradients will
#         not be clipped
#     epsilon :: float - fuzz factor
#     gradient_moment :: numpy.ndarray - running optimization variable
#     gradient_square_moment :: numpy.ndarray - running optimization variable
#     initial_learning_rate :: float - the initial step size
#     iteration_count :: int - the current count of iterations performed
#     learning_rate :: float - the current step size
#     learning_rate_decay :: float - the number of iterations it takes for
#         the learning rate to decay by 1/e, if not set, no decay is
#         applied
#     operation_policy
#     name :: str - identifier for the optimizer
#     scale_grads :: float - the value to scale the norm of the gradients to,
#         if not set, the gradients will not be scaled
def main():
    result = grape_schroedinger_discrete(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                         COSTS, EVOLUTION_TIME, hamiltonian,
                                         INITIAL_STATES, SYSTEM_EVAL_COUNT,
                                         complex_controls=False,
                                    cost_eval_step=1,
                                    impose_control_conditions=None,
                                    initial_controls=None,
                                    interpolation_policy=InterpolationPolicy.LINEAR,
                                    iteration_count=1000,
                                    log_iteration_step=10,
                                    magnus_policy=MagnusPolicy.M2,
                                    max_control_norms=None,
                                    min_error=0,
                                    optimizer=optimizer,
                                    save_file_path=None,
                                    save_intermediate_states=False,
                                    save_iteration_step=0,manual_parameter=manual_parameter)
#
#Args:
#    control_count :: int - This is the number of control parameters that qoc should
#        optimize over. I.e. it is the length of the `controls` array passed
#        to the hamiltonian.
#    control_eval_count :: int >= 2 - This value determines where definite values
#        of the control parameters are evaluated. This value is used as:
#        `control_eval_times`= numpy.linspace(0, `evolution_time`, `control_eval_count`).
#   costs :: iterable(qoc.models.cost.Cost) - This list specifies all
#       the cost functions that the optimizer should evaluate. This list
#        defines the criteria for an "optimal" control set.
#    evolution_time :: float - This value specifies the duration of the
#        system's evolution.
#    hamiltonian :: (controls :: ndarray (control_count), time :: float)
#                   -> hamiltonian_matrix :: ndarray (hilbert_size x hilbert_size)
#        - This function provides the system's hamiltonian given a set
#        of control parameters and a time value.
#    initial_states :: ndarray (state_count x hilbert_size x 1)
#        - This array specifies the states that should be evolved under the
#        specified system. These are the states at the beginning of the evolution.
#    system_eval_count :: int >= 2 - This value determines how many times
#        during the evolution the system is evaluated, including the
#        initial value of the system. For the schroedinger evolution,
#        this value determines the time step of integration.
#        This value is used as:
#        `system_eval_times` = numpy.linspace(0, `evolution_time`, `system_eval_count`).
#
#    complex_controls :: bool - This value determines if the control parameters
#        are complex-valued. If some controls are real only or imaginary only
#        while others are complex, real only and imaginary only controls
#        can be simulated by taking the real or imaginary part of a complex control.
#    cost_eval_step :: int >= 1- This value determines how often step-costs are evaluated.
#         The units of this value are in system_eval steps. E.g. if this value is 2,
#         step-costs will be computed every 2 system_eval steps.
#    impose_control_conditions :: (controls :: (control_eval_count x control_count))
#                                -> (controls :: (control_eval_count x control_count))
#        - This function is called after every optimization update. Example uses
#        include setting boundary conditions on the control parameters.                             
#    initial_controls :: ndarray (control_step_count x control_count)
#       - This array specifies the control parameters at each
#        control step. These values will be used to determine the `controls`
#        argument passed to the `hamiltonian` function at each time step for
#        the first iteration of optimization.
#    interpolation_policy :: qoc.models.interpolationpolicy.InterpolationPolicy
#        - This value specifies how control parameters should be
#        interpreted at points where they are not defined.
#    iteration_count :: int - This value determines how many total system
#        evolutions the optimizer will perform to determine the
#       optimal control set.
#    log_iteration_step :: int - This value determines how often qoc logs
#        progress to stdout. This value is specified in units of system steps,
#        of which there are `control_step_count` * `system_step_multiplier`.
#        Set this value to 0 to disable logging.
#    magnus_policy :: qoc.models.magnuspolicy.MagnusPolicy - This value
#        specifies what method should be used to perform the magnus expansion
#        of the system matrix for ode integration. Choosing a higher order
#        magnus expansion will yield more accuracy, but it will
#        result in a longer compute time.
#    max_control_norms :: ndarray (control_count) - This array
#       specifies the element-wise maximum norm that each control is
#        allowed to achieve. If, in optimization, the value of a control
#        exceeds its maximum norm, the control will be rescaled to
#        its maximum norm. Note that for non-complex values, this
#        feature acts exactly as absolute value clipping.
#    min_error :: float - This value is the threshold below which
#        optimization will terminate.
#    optimizer :: class instance - This optimizer object defines the
#        gradient-based procedure for minimizing the total contribution
#        of all cost functions with respect to the control parameters.
#    save_file_path :: str - This is the full path to the file where
#        information about program execution will be stored.
#        E.g. "./out/foo.h5"
#    save_intermediate_densities :: bool - If this value is set to True,
#        qoc will write the densities to the save file after every
#        system_eval step.
#    save_intermediate_states :: bool - If this value is set to True,
#        qoc will write the states to the save file after every
#        system_eval step.
#    save_iteration_step :: int - This value determines how often qoc
#        saves progress to the save file specified by `save_file_path`.
#        This value is specified in units of system steps, of which
#       there are `control_step_count` * `system_step_multiplier`.
#        Set this value to 0 to disable saving.
        
if __name__ == "__main__":
    main()




False
iter   |   total error  |    grads_l2   
  0    | 9.78697344e-01 | 3.53190944e-01
  10   | 9.62152856e-01 | 4.77545945e-01
  20   | 9.39886294e-01 | 5.91627441e-01
  30   | 9.11757318e-01 | 6.90381060e-01
  40   | 8.78377852e-01 | 7.66530000e-01
  50   | 8.41231421e-01 | 8.13572513e-01
  60   | 8.02463194e-01 | 8.28174726e-01
  70   | 7.64448944e-01 | 8.11437601e-01
  80   | 7.29302665e-01 | 7.68743084e-01
  90   | 6.98487899e-01 | 7.08377158e-01
 100   | 6.72647648e-01 | 6.39517178e-01
 110   | 6.51673708e-01 | 5.70321921e-01
 120   | 6.34941983e-01 | 5.06695123e-01
 130   | 6.21594882e-01 | 4.51916448e-01
 140   | 6.10771059e-01 | 4.06976741e-01
 150   | 6.01739113e-01 | 3.71281227e-01
 160   | 5.93943442e-01 | 3.43399960e-01
 170   | 5.86994430e-01 | 3.21660991e-01
 180   | 5.80634541e-01 | 3.04514122e-01
 190   | 5.74700774e-01 | 2.90691072e-01
 200   | 5.69092979e-01 | 2.79231579e-01
 210   | 5.63750525e-01 | 2.69443431e-01
 220   | 5.58636555e-01 | 2.60842396e-01
 230   | 5

# Cost for forbidden states

In [4]:


import numpy as np

from qoc.core import grape_schroedinger_discrete
from qoc.standard import (ForbidStates, SIGMA_X, SIGMA_Y,)
hilbert_size = 4
hamiltonian_matrix = np.divide(1, 2) * (np.kron(SIGMA_X, SIGMA_X)
                                            + np.kron(SIGMA_Y, SIGMA_Y))
hamiltonian = lambda controls, t: (controls[0] * hamiltonian_matrix)
initial_states = np.array([[[0], [1], [0], [0]]])
forbidden_states = np.array([[[[0], [1], [0], [0]]]])
control_count = 1
evolution_time = 10
control_eval_count = system_eval_count = 11
costs = [ForbidStates(forbidden_states=forbidden_states, system_eval_count=system_eval_count,cost_eval_step=1,
                 cost_multiplier=1.,)]
iteration_count = 1000
log_iteration_step = 1
manual_parameter={"control_hamiltonian":[hamiltonian_matrix],"manual_gradient_mode":False}
def main():
    result = grape_schroedinger_discrete(control_count, control_eval_count,
                                         costs, evolution_time,
                                         hamiltonian, initial_states,
                                         system_eval_count,
                                         iteration_count=iteration_count,
                                         log_iteration_step=log_iteration_step,
                                         manual_parameter=manual_parameter
                                         )
if __name__ == "__main__":
    main()

iter   |   total error  |    grads_l2   
  0    | 6.91162432e-01 | 1.62820647e+00
  1    | 6.86353720e-01 | 1.62888413e+00
  2    | 6.81546931e-01 | 1.62920640e+00
  3    | 6.76743260e-01 | 1.62917491e+00
  4    | 6.71943924e-01 | 1.62879144e+00
  5    | 6.67150164e-01 | 1.62805793e+00
  6    | 6.62363242e-01 | 1.62697649e+00
  7    | 6.57584445e-01 | 1.62554942e+00
  8    | 6.52815075e-01 | 1.62377922e+00
  9    | 6.48056455e-01 | 1.62166858e+00
  10   | 6.43309924e-01 | 1.61922041e+00
  11   | 6.38576836e-01 | 1.61643784e+00
  12   | 6.33858555e-01 | 1.61332423e+00
  13   | 6.29156457e-01 | 1.60988315e+00
  14   | 6.24471923e-01 | 1.60611842e+00
  15   | 6.19806342e-01 | 1.60203412e+00
  16   | 6.15161102e-01 | 1.59763455e+00
  17   | 6.10537593e-01 | 1.59292426e+00
  18   | 6.05937198e-01 | 1.58790806e+00
  19   | 6.01361297e-01 | 1.58259100e+00
  20   | 5.96811258e-01 | 1.57697839e+00
  21   | 5.92288438e-01 | 1.57107576e+00
  22   | 5.87794179e-01 | 1.56488891e+00
  23   | 5.83329

 199   | 2.75379325e-01 | 4.88994971e-01
 200   | 2.74648623e-01 | 4.87575447e-01
 201   | 2.73920587e-01 | 4.86168515e-01
 202   | 2.73195209e-01 | 4.84773833e-01
 203   | 2.72472482e-01 | 4.83391066e-01
 204   | 2.71752397e-01 | 4.82019890e-01
 205   | 2.71034948e-01 | 4.80659990e-01
 206   | 2.70320128e-01 | 4.79311061e-01
 207   | 2.69607930e-01 | 4.77972805e-01
 208   | 2.68898348e-01 | 4.76644935e-01
 209   | 2.68191376e-01 | 4.75327171e-01
 210   | 2.67487008e-01 | 4.74019241e-01
 211   | 2.66785238e-01 | 4.72720881e-01
 212   | 2.66086061e-01 | 4.71431837e-01
 213   | 2.65389471e-01 | 4.70151860e-01
 214   | 2.64695463e-01 | 4.68880709e-01
 215   | 2.64004032e-01 | 4.67618151e-01
 216   | 2.63315173e-01 | 4.66363961e-01
 217   | 2.62628881e-01 | 4.65117917e-01
 218   | 2.61945151e-01 | 4.63879808e-01
 219   | 2.61263979e-01 | 4.62649428e-01
 220   | 2.60585359e-01 | 4.61426577e-01
 221   | 2.59909289e-01 | 4.60211060e-01
 222   | 2.59235762e-01 | 4.59002690e-01
 223   | 2.58564

 400   | 1.74115787e-01 | 2.97804043e-01
 401   | 1.73791741e-01 | 2.97123163e-01
 402   | 1.73468926e-01 | 2.96444665e-01
 403   | 1.73147335e-01 | 2.95768547e-01
 404   | 1.72826959e-01 | 2.95094805e-01
 405   | 1.72507793e-01 | 2.94423435e-01
 406   | 1.72189828e-01 | 2.93754433e-01
 407   | 1.71873057e-01 | 2.93087796e-01
 408   | 1.71557472e-01 | 2.92423520e-01
 409   | 1.71243067e-01 | 2.91761602e-01
 410   | 1.70929834e-01 | 2.91102036e-01
 411   | 1.70617766e-01 | 2.90444819e-01
 412   | 1.70306856e-01 | 2.89789947e-01
 413   | 1.69997097e-01 | 2.89137415e-01
 414   | 1.69688481e-01 | 2.88487220e-01
 415   | 1.69381002e-01 | 2.87839356e-01
 416   | 1.69074652e-01 | 2.87193819e-01
 417   | 1.68769424e-01 | 2.86550605e-01
 418   | 1.68465312e-01 | 2.85909708e-01
 419   | 1.68162309e-01 | 2.85271125e-01
 420   | 1.67860408e-01 | 2.84634850e-01
 421   | 1.67559602e-01 | 2.84000878e-01
 422   | 1.67259884e-01 | 2.83369204e-01
 423   | 1.66961247e-01 | 2.82739824e-01
 424   | 1.66663

 600   | 1.26380794e-01 | 2.01008189e-01
 601   | 1.26201762e-01 | 2.00675301e-01
 602   | 1.26023148e-01 | 2.00343459e-01
 603   | 1.25844951e-01 | 2.00012659e-01
 604   | 1.25667170e-01 | 1.99682894e-01
 605   | 1.25489802e-01 | 1.99354160e-01
 606   | 1.25312846e-01 | 1.99026450e-01
 607   | 1.25136301e-01 | 1.98699760e-01
 608   | 1.24960165e-01 | 1.98374084e-01
 609   | 1.24784436e-01 | 1.98049416e-01
 610   | 1.24609113e-01 | 1.97725752e-01
 611   | 1.24434195e-01 | 1.97403085e-01
 612   | 1.24259679e-01 | 1.97081411e-01
 613   | 1.24085565e-01 | 1.96760725e-01
 614   | 1.23911851e-01 | 1.96441021e-01
 615   | 1.23738535e-01 | 1.96122294e-01
 616   | 1.23565617e-01 | 1.95804539e-01
 617   | 1.23393094e-01 | 1.95487751e-01
 618   | 1.23220965e-01 | 1.95171925e-01
 619   | 1.23049229e-01 | 1.94857055e-01
 620   | 1.22877884e-01 | 1.94543137e-01
 621   | 1.22706929e-01 | 1.94230166e-01
 622   | 1.22536362e-01 | 1.93918137e-01
 623   | 1.22366183e-01 | 1.93607045e-01
 624   | 1.22196

 806   | 9.66198562e-02 | 1.48617423e-01
 807   | 9.65037032e-02 | 1.48421509e-01
 808   | 9.63877718e-02 | 1.48226017e-01
 809   | 9.62720614e-02 | 1.48030948e-01
 810   | 9.61565715e-02 | 1.47836299e-01
 811   | 9.60413013e-02 | 1.47642069e-01
 812   | 9.59262502e-02 | 1.47448256e-01
 813   | 9.58114177e-02 | 1.47254860e-01
 814   | 9.56968031e-02 | 1.47061879e-01
 815   | 9.55824058e-02 | 1.46869312e-01
 816   | 9.54682252e-02 | 1.46677157e-01
 817   | 9.53542607e-02 | 1.46485414e-01
 818   | 9.52405116e-02 | 1.46294079e-01
 819   | 9.51269774e-02 | 1.46103154e-01
 820   | 9.50136574e-02 | 1.45912635e-01
 821   | 9.49005511e-02 | 1.45722523e-01
 822   | 9.47876578e-02 | 1.45532814e-01
 823   | 9.46749769e-02 | 1.45343510e-01
 824   | 9.45625079e-02 | 1.45154607e-01
 825   | 9.44502501e-02 | 1.44966105e-01
 826   | 9.43382030e-02 | 1.44778002e-01
 827   | 9.42263659e-02 | 1.44590298e-01
 828   | 9.41147382e-02 | 1.44402991e-01
 829   | 9.40033194e-02 | 1.44216079e-01
 830   | 9.38921

# Cost for controls

In [2]:
"""

"""



from qoc import grape_schroedinger_discrete
from qoc.standard import (TargetStateInfidelity,
                          conjugate_transpose,
                          get_annihilation_operator,
                          get_creation_operator,
                          SIGMA_Z,
                          generate_save_file_path, ForbidStates, ControlArea, ControlNorm, ControlVariation,
                          ControlBandwidthMax,)
import numpy as np

# Define the system.
HILBERT_SIZE = 2
ANNIHILATION_OPERATOR = get_annihilation_operator(HILBERT_SIZE)
CREATION_OPERATOR = get_creation_operator(HILBERT_SIZE)
sigmax=ANNIHILATION_OPERATOR +CREATION_OPERATOR
sigmay=-1j*ANNIHILATION_OPERATOR+1j*CREATION_OPERATOR
# E.q. 19 (p. 6) of https://arxiv.org/abs/1904.06560.
H_SYSTEM_0 = SIGMA_Z / 2
# Use a + a^{dagger} as the drive term to control.
hamiltonian = lambda controls, time: (H_SYSTEM_0
                                      + controls[0] * sigmax
                                      + controls[1]*sigmay)
CONTROL_HAMILTONIAN=[sigmax,sigmay]
# Define the problem.
INITIAL_STATE_0 = np.array([[1], [0]])
TARGET_STATE_0 = np.array([[0], [1]])
INITIAL_STATES = np.stack((INITIAL_STATE_0,), axis=0)
TARGET_STATES = np.stack((TARGET_STATE_0,), axis=0)
FORBIDDEN_STATES = np.stack((INITIAL_STATES,), axis=0)

# Define the optimization.
COMPLEX_CONTROLS = False
CONTROL_COUNT = 2
EVOLUTION_TIME = 10 # nanoseconds
CONTROL_EVAL_COUNT  = 5
SYSTEM_EVAL_COUNT=5
ITERATION_COUNT = 1000
max_norm = 2
initial_controls = np.array([[0.1,0.1],[0.2,0.2],[0.3,0.3],[0.4,0.4],[0.5,0.5]])

# Define output.
LOG_ITERATION_STEP = 1
SAVE_ITERATION_STEP = 1
SAVE_PATH = "./out"
SAVE_FILE_NAME = "transmon_pi"
SAVE_FILE_PATH = generate_save_file_path(SAVE_FILE_NAME, SAVE_PATH)
manual_parameter={"control_hamiltonian":CONTROL_HAMILTONIAN,"manual_gradient_mode":False}


# This cost penalizes control frequencies above a set maximum.

#     Fields:
#     max_bandwidths :: ndarray (CONTROL_COUNT) - This array contains the maximum allowed bandwidth of each control.
#     control_count
#     freqs :: ndarray (CONTROL_EVAL_COUNT) - This array contains the frequencies of each of the controls.
#     name
#     requires_step_evaluation
MAX_BANDWIDTH_0 = 0.001 # GHz
MAX_BANDWIDTHS = np.array((MAX_BANDWIDTH_0,MAX_BANDWIDTH_0))
cost1=ControlBandwidthMax(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                 EVOLUTION_TIME, MAX_BANDWIDTHS)

# This cost penalizes the area under the
#     function of time generated by the discrete control parameters.
max_control_norms=0.001
cost2=ControlArea(control_count=CONTROL_COUNT,
                 control_eval_count=CONTROL_EVAL_COUNT,
                 cost_multiplier=1.,
                 max_control_norms=max_control_norms,)



# This cost penalizes the value of the norm of the control parameters.

#     Fields:
#     control_weights :: ndarray (control_eval_count x control_count)
#         - These weights, each of which should be no greater than 1,
#         represent the factor by which each control's magnitude is penalized.
#         If no weights are specified, each control's magnitude is penalized
#         equally.
cost3=ControlNorm(control_count=CONTROL_COUNT,
                 control_eval_count=CONTROL_EVAL_COUNT,
                 control_weights=None,
                 cost_multiplier=1.,
                 max_control_norms=None)
# This cost penalizes the variations of the control parameters
#     from one `control_eval_step` to the next.
# manual only supports first and second order so far.
cost4=ControlVariation(control_count=CONTROL_COUNT,
                 control_eval_count=CONTROL_EVAL_COUNT,
                 cost_multiplier=1.,
                 max_control_norms=None,
                 order=1)

COSTS = [cost2,cost1]
def main():
    result = grape_schroedinger_discrete(CONTROL_COUNT, CONTROL_EVAL_COUNT,
                                         COSTS, EVOLUTION_TIME, hamiltonian,
                                         INITIAL_STATES, SYSTEM_EVAL_COUNT,
                                         initial_controls=initial_controls,
                                         complex_controls=COMPLEX_CONTROLS,
                                         iteration_count=ITERATION_COUNT,
                                         log_iteration_step=LOG_ITERATION_STEP,
                                         save_file_path=SAVE_FILE_PATH,
                                         save_iteration_step=SAVE_ITERATION_STEP,
                                         manual_parameter=manual_parameter
                                         )


if __name__ == "__main__":
    main()


QOC is saving this optimization run to ./out/00013_transmon_pi.h5.
iter   |   total error  |    grads_l2   
  0    | 3.00809017e+02 | 3.16231541e+02
  1    | 2.99809017e+02 | 3.16231541e+02
  2    | 2.98809017e+02 | 3.16231541e+02
  3    | 2.97809017e+02 | 3.16231541e+02
  4    | 2.96809017e+02 | 3.16231541e+02
  5    | 2.95809017e+02 | 3.16231541e+02
  6    | 2.94809017e+02 | 3.16231541e+02
  7    | 2.93809017e+02 | 3.16231541e+02
  8    | 2.92809017e+02 | 3.16231541e+02
  9    | 2.91809017e+02 | 3.16231541e+02
  10   | 2.90809017e+02 | 3.16231541e+02
  11   | 2.89809017e+02 | 3.16231541e+02
  12   | 2.88809017e+02 | 3.16231541e+02
  13   | 2.87809017e+02 | 3.16231541e+02
  14   | 2.86809017e+02 | 3.16231541e+02
  15   | 2.85809017e+02 | 3.16231541e+02
  16   | 2.84809017e+02 | 3.16231541e+02
  17   | 2.83809017e+02 | 3.16231541e+02
  18   | 2.82809017e+02 | 3.16231541e+02
  19   | 2.81809017e+02 | 3.16231541e+02
  20   | 2.80809017e+02 | 3.16231541e+02
  21   | 2.79809017e+02 | 3.162

 197   | 1.03809017e+02 | 3.16231541e+02
 198   | 1.02809017e+02 | 3.16231541e+02
 199   | 1.01809017e+02 | 3.16231541e+02
 200   | 1.00809017e+02 | 3.16231541e+02
 201   | 9.98090170e+01 | 3.16231541e+02
 202   | 9.88090170e+01 | 3.16231541e+02
 203   | 9.78090170e+01 | 3.16231541e+02
 204   | 9.68090170e+01 | 3.16231541e+02
 205   | 9.58090170e+01 | 3.16231541e+02
 206   | 9.48090170e+01 | 3.16231541e+02
 207   | 9.38090170e+01 | 3.16231541e+02
 208   | 9.28090170e+01 | 3.16231541e+02
 209   | 9.18090170e+01 | 3.16231541e+02
 210   | 9.08090170e+01 | 3.16231541e+02
 211   | 8.98090170e+01 | 3.16231541e+02
 212   | 8.88090170e+01 | 3.16231541e+02
 213   | 8.78090170e+01 | 3.16231541e+02
 214   | 8.68090170e+01 | 3.16231541e+02
 215   | 8.58090170e+01 | 3.16231541e+02
 216   | 8.48090170e+01 | 3.16231541e+02
 217   | 8.38090170e+01 | 3.16231541e+02
 218   | 8.28090170e+01 | 3.16231541e+02
 219   | 8.18090170e+01 | 3.16231541e+02
 220   | 8.08090170e+01 | 3.16231541e+02
 221   | 7.98090

 401   | 1.22650291e+00 | 3.16231509e+02
 402   | 1.28428429e+00 | 3.16231508e+02
 403   | 1.23628156e+00 | 3.16231508e+02
 404   | 1.09307321e+00 | 3.16231508e+02
 405   | 8.64179874e-01 | 3.16231507e+02
 406   | 1.05483338e+00 | 3.16231507e+02
 407   | 1.23019126e+00 | 3.16231507e+02
 408   | 1.28801378e+00 | 3.16231506e+02
 409   | 1.24005448e+00 | 3.16231506e+02
 410   | 1.09689153e+00 | 3.16231506e+02
 411   | 8.68045302e-01 | 3.16231505e+02
 412   | 1.05065118e+00 | 3.16231505e+02
 413   | 1.22597652e+00 | 3.16231505e+02
 414   | 1.28376351e+00 | 3.16231504e+02
 415   | 1.23576604e+00 | 3.16231504e+02
 416   | 1.09256266e+00 | 3.16231504e+02
 417   | 8.63674015e-01 | 3.16231503e+02
 418   | 1.05476784e+00 | 3.16231503e+02
 419   | 1.23012208e+00 | 3.16231503e+02
 420   | 1.28794133e+00 | 3.16231502e+02
 421   | 1.23997908e+00 | 3.16231502e+02
 422   | 1.09681347e+00 | 3.16231502e+02
 423   | 8.67964856e-01 | 3.16231502e+02
 424   | 1.05016655e+00 | 3.16231501e+02
 425   | 1.22549

 602   | 1.09477434e+00 | 3.16231446e+02
 603   | 8.65920371e-01 | 3.16231446e+02
 604   | 1.04377586e+00 | 3.16231446e+02
 605   | 1.21910718e+00 | 3.16231445e+02
 606   | 1.27690174e+00 | 3.16231445e+02
 607   | 1.22891327e+00 | 3.16231445e+02
 608   | 1.08572012e+00 | 3.16231445e+02
 609   | 8.56842771e-01 | 3.16231444e+02
 610   | 1.05258977e+00 | 3.16231444e+02
 611   | 1.22793671e+00 | 3.16231444e+02
 612   | 1.28574916e+00 | 3.16231443e+02
 613   | 1.23778056e+00 | 3.16231443e+02
 614   | 1.09460902e+00 | 3.16231443e+02
 615   | 8.65754833e-01 | 3.16231442e+02
 616   | 1.04338341e+00 | 3.16231442e+02
 617   | 1.21871490e+00 | 3.16231442e+02
 618   | 1.27650971e+00 | 3.16231442e+02
 619   | 1.22852155e+00 | 3.16231441e+02
 620   | 1.08532874e+00 | 3.16231441e+02
 621   | 8.56451786e-01 | 3.16231441e+02
 622   | 1.05242263e+00 | 3.16231440e+02
 623   | 1.22776935e+00 | 3.16231440e+02
 624   | 1.28558157e+00 | 3.16231440e+02
 625   | 1.23761275e+00 | 3.16231439e+02
 626   | 1.09444

 808   | 1.03747911e+00 | 3.16231385e+02
 809   | 1.21281293e+00 | 3.16231385e+02
 810   | 1.27061075e+00 | 3.16231385e+02
 811   | 1.22262619e+00 | 3.16231384e+02
 812   | 1.07943752e+00 | 3.16231384e+02
 813   | 8.50565164e-01 | 3.16231384e+02
 814   | 1.04945408e+00 | 3.16231383e+02
 815   | 1.22479827e+00 | 3.16231383e+02
 816   | 1.28260794e+00 | 3.16231383e+02
 817   | 1.23463656e+00 | 3.16231382e+02
 818   | 1.09146222e+00 | 3.16231382e+02
 819   | 8.62605232e-01 | 3.16231382e+02
 820   | 1.03712943e+00 | 3.16231382e+02
 821   | 1.21246337e+00 | 3.16231381e+02
 822   | 1.27026133e+00 | 3.16231381e+02
 823   | 1.22227694e+00 | 3.16231381e+02
 824   | 1.07908846e+00 | 3.16231380e+02
 825   | 8.50216320e-01 | 3.16231380e+02
 826   | 1.04925406e+00 | 3.16231380e+02
 827   | 1.22459814e+00 | 3.16231380e+02
 828   | 1.28240771e+00 | 3.16231379e+02
 829   | 1.23443621e+00 | 3.16231379e+02
 830   | 1.09126175e+00 | 3.16231379e+02
 831   | 8.62404644e-01 | 3.16231378e+02
 832   | 1.03678